In [6]:
# Commented out IPython magic to ensure Python compatibility.
# %%capture
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import keras
import keras.backend as K

from sklearn.model_selection import ParameterGrid
from py_vollib import black_scholes_merton as bsm
from progressbar import ProgressBar
from scipy.stats import gamma
from scipy.stats import beta
from scipy.stats import uniform
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

In [8]:
# S (spot price)
# gamma
def thisS(q):
    return gamma.ppf(q, a = 100, scale = 1)

# K (strike price)
# uniform (lower = 50, upper = 200)
def thisK(q):
    return uniform.ppf(q, 50, 200)

# (interest rate)
# uniform (lower = 0.01, upper = 0.18)
def thisR(q):
    return uniform.ppf(q, 0.01, 0.18)

# D (dividend)
# uniform (lower = 0.01, upper = 0.18)
def thisD(q):
    return uniform.ppf(q, 0.01, 0.18)

# t (time-to-maturity)
# t will be 3, 6, 9, 12 months for all examples (0.25, 0.5, 0.75, 1 year)

# sigma (volatility)
# beta (add small amount so volatility cannot be zero)
def thisSigma(q):
    return (beta.ppf(q, a = 2, b = 5) + 0.001)

In [12]:
num_increment = 12
percentiles = pd.Series(np.linspace(0, 0.99, num_increment))

S = percentiles.apply(thisS)
K = percentiles.apply(thisK)
q = percentiles.apply(thisD)
t = np.array([.25, .5, .75, 1])
r = percentiles.apply(thisR)
sigma = percentiles.apply(thisSigma)

param_grid = {'S': S, 'K': K, 'q': q, 't': t, 'r': r, 'sigma': sigma}
grid = ParameterGrid(param_grid)
pbar = ProgressBar()
fullDF = pd.DataFrame()
prices = []
for params in pbar(grid):
    prices.append(bsm.black_scholes_merton(flag = 'p', S = params['S'], K = params['K'],\
    q = params['q'], t = params['t'],r = params['r'], sigma = params['sigma']))
    fullDF = fullDF.append(pd.Series(params), ignore_index = True)
    pass

# swap price to first column
fullDF['price'] = prices

# output to csv
fullDF.to_csv('dataFull.csv', index = False)
print(fullDF.head())
print(fullDF.tail())
num_increment = 5
percentiles = pd.Series(np.linspace(0, 0.99, num_increment))

S = percentiles.apply(thisS)
K = percentiles.apply(thisK)
q = percentiles.apply(thisD)
t = np.array([.25, .5, .75, 1])
r = percentiles.apply(thisR)
sigma = percentiles.apply(thisSigma)

param_grid = {'S': S, 'K' : K, 'q' : q, 't' : t, 'r' : r, 'sigma' : sigma}
grid = ParameterGrid(param_grid)

TypeError: Parameter grid for parameter 'S' needs to be a list or a numpy array, but got 0       0.000000
1      86.877620
2      90.808086
3      93.675159
4      96.131942
5      98.417953
6     100.672612
7     103.015743
8     105.597693
9     108.687444
10    113.010524
11    124.722561
dtype: float64 (of type Series) instead. Single values need to be wrapped in a list with one element.

In [ ]:
pbar = ProgressBar()
sparseDF = pd.DataFrame()
prices = []
for params in pbar(grid):
    prices.append(bsm.black_scholes_merton(flag = 'p', S = params['S'], K = params['K'],\
    q = params['q'], t = params['t'],r = params['r'], sigma = params['sigma']))
    sparseDF = sparseDF.append(pd.Series(params), ignore_index = True)

# swap price to first column
sparseDF['price'] = prices

# output to csv
sparseDF.to_csv('dataSparse.csv', index = False)
print(sparseDF.head())
print(sparseDF.tail())

def this_extremes_S (q):
    return uniform.ppf(q, 90, 110)
S = percentiles.apply(this_extremes_S)

K = percentiles.apply(thisK)
q = percentiles.apply(thisD)
t = np.array([.25, .5, .75, 1])
r = percentiles.apply(thisR)
sigma = percentiles.apply(thisSigma)

param_grid = {'S': S, 'K' : K, 'q' : q, 't' : t, 'r' : r, 'sigma' : sigma}
grid = ParameterGrid(param_grid)

pbar = ProgressBar()
extremesDF = pd.DataFrame()
prices = []
for params in pbar(grid):
prices.append(bsm.black_scholes_merton(flag = 'p', S = params['S'], K = params['K'],\
q = params['q'], t = params['t'],r = params['r'], sigma = params['sigma']))
extremesDF = extremesDF.append(pd.Series(params), ignore_index = True)

# swap price to first column
extremesDF['price'] = prices

# output to csv
extremesDF.to_csv('dataExtremes.csv', index = False)
print(extremesDF.head())
print(extremesDF.tail())

# testing neural network (full data)
fullDF = pd.read_csv("dataFullMain.csv")

In [ ]:
def baseline_model():
    # create model
    i = Input(shape=(6,))
    x = Dense(10, activation='relu')(i)
    y = Dense(10, activation='relu')(x)
    o = Dense(1)(y)
    model = Model(i, o)
    model.compile(loss="mse", optimizer= "adam")
    return model

model_full = baseline_model()
X = fullDF[['S','K','q','r','sigma','t']]
y = fullDF[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 7)
history_full = model_full.fit(X_train, y_train, batch_size = 64, epochs = 20, \
verbose = 2, validation_split=0.2) # set batch size to 1, otherwise there are errors when trying to

plt.plot(history_full.history['val_loss'])
plt.title('Model validation loss')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.legend(['Error', 'Test'], loc='upper left')
plt.show()
X_test_full = X_test
y_test_full = y_test
model_full.evaluate(x=X_test, y=y_test)

# testing neural network (sparse data)

sparseDF = pd.read_csv("dataSparseMain.csv")

def baseline_model():
    # create model
    i = Input(shape=(6,))
    x = Dense(10, activation='relu')(i)
    y = Dense(10, activation='relu')(x)
    o = Dense(1)(y)
    model = Model(i, o)
    model.compile(loss="mse", optimizer= "adam")
    return model

model_sparse = baseline_model()
X = sparseDF[['S','K','q','r','sigma','t']]
y = sparseDF[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 7)
history_sparse = model_sparse.fit(X_train, y_train, batch_size = 64, epochs = 20, \
verbose = 2, validation_split=0.2) # set batch size to 1, otherwise there are errors when trying to

plt.plot(history_sparse.history['val_loss'])
plt.title('Model validation loss')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.legend(['Error', 'Test'], loc='upper left')
plt.show()

In [ ]:
model_sparse.evaluate(x=X_test_full, y=y_test_full)

# testing neural network (extremes data)
extremesDF = pd.read_csv("dataExtremesMain.csv")

def baseline_model():
    # create model
    i = Input(shape=(6,))
    x = Dense(10, activation='relu')(i)
    y = Dense(10, activation='relu')(x)
    o = Dense(1)(y)
    model = Model(i, o)
    model.compile(loss="mse", optimizer= "adam")
    return model

model_extremes = baseline_model()
X = extremesDF[['S','K','q','r','sigma','t']]
y = extremesDF[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 7)
history_extremes = model_extremes.fit(X_train, y_train, batch_size = 64, epochs = 20, \
verbose = 2, validation_split=0.2) # set batch size to 1, otherwise there are errors when trying to

plt.plot(history_extremes.history['val_loss'])
plt.title('Model validation loss')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.legend(['Error', 'Test'], loc='upper left')
plt.show()

model_extremes.evaluate(x=X_test_full, y=y_test_full)

tableOutput = pd.DataFrame({'Full':history_full.history['val_loss'], \
'Sparse':history_sparse.history['val_loss'], \
'Extremes':history_extremes.history['val_loss']}, columns=['Full', 'Sparse', 'Extremes'])
tableOutput.to_csv("tableResultsValidaton.csv")

print(len(fullDF.index))
print(len(sparseDF.index))
print(len(extremesDF.index))